# What I want to do with this
##                                                                EDA setup
I would like to take the average household statistics from the range of bedrooms, bathrooms, zipcodes etc and adjust the average of all the zeros that show up in those ranges with the averages of those areas. Possibly change the waterfront views that are set to zero in that are near the lat's and long's as well from zeros and put them as ones to better represent the data a little more accurately. After taking a look at that and making sure thats acceptable I will start by setting a function to graph the data on a map.(Don't take too long on this as it is a decent representation of the information it is also not needed).
##                                                    Interesting representation
Possible representation of the data could be through zipcode and bin them in with each other and set them out as zones as well. Could take a look at the 'Areas' as a way to make sense of the data and the pricing of each. could also set up a function to pull specific information for the conditions of what a particular pearson wants as well as a budget. Taking note from both House Hunters and DIY shows on what it is that could be done with deemable 'Fixer Uppers' with a certain budget. Set that function towards anything below their budget and whether they want to fix a place up or not. If time alloted I will use the map made earlier to graph the places and show where those places are adjesent to important places like hospitals, schools, possibly shopping centers, etc. Could also figure a way to find out if the zones have crime rates etc around them and deem if it is a safe neighborhood.

##                                                   Statistical representation
Train a model to take in the account of stuff the person is looking at and adjust the amount of money they would spend in "upgrading" their home and how much they would expect in return for their investments if they were to sell their house, refinance etc and if it would be deemed acceptible if they want to flip the house how much they would be able to sell later on.                                            



# Exploring the data
First things first, checking and fixing up the data. I am looking for anything that doesn't belong like NaN's and irrelevent data

In [1]:
# !pip install arcgis
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import GeoAccessor,GeoSeriesAccessor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('data/kc_house_data.csv')

## Checking the dataframe
Taking a look at what the dataframe comprises of and taking a look at what irrelevent data is there

In [2]:
df.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
display(df.isna().sum())
display(df.max())
df.min()
df.price.astype(int)


id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

id               9900000190
date               9/9/2014
price               7.7e+06
bedrooms                 33
bathrooms                 8
sqft_living           13540
sqft_lot            1651359
floors                  3.5
waterfront                1
view                      4
condition                 5
grade                    13
sqft_above             9410
sqft_basement             ?
yr_built               2015
yr_renovated           2015
zipcode               98199
lat                 47.7776
long               -121.315
sqft_living15          6210
sqft_lot15           871200
dtype: object

0        221900
1        538000
2        180000
3        604000
4        510000
          ...  
21592    360000
21593    400000
21594    402101
21595    400000
21596    325000
Name: price, Length: 21597, dtype: int32

## Outliers
it seems that there are a few missing data and some outliers. The max shows a bedroom capacity of 33, will take a look at the here.

In [4]:
df[df['bedrooms'] == 33]
# wouldn't be realistic for a house with 33 bedrooms to only have a sqft_living of 1620 and only 1 3/4 bathrooms so it looks like a typo
# will adjust to 3 
df[df['bedrooms'] == 33] = df[df['bedrooms'] == 33].replace(33,3)
df['sqft_basement'] = df['sqft_basement'].replace('?','0').astype(float)
df['waterfront'] = df['waterfront'].fillna(0)
df['view'] = df['view'].fillna(0)
# should remove id, lat,long from dataframe to work with median 
df1 = df.drop('id',1).drop('lat',1).drop('long',1)

In [5]:
df.info

<bound method DataFrame.info of                id        date     price  bedrooms  bathrooms  sqft_living  \
0      7129300520  10/13/2014  221900.0         3       1.00         1180   
1      6414100192   12/9/2014  538000.0         3       2.25         2570   
2      5631500400   2/25/2015  180000.0         2       1.00          770   
3      2487200875   12/9/2014  604000.0         4       3.00         1960   
4      1954400510   2/18/2015  510000.0         3       2.00         1680   
...           ...         ...       ...       ...        ...          ...   
21592   263000018   5/21/2014  360000.0         3       2.50         1530   
21593  6600060120   2/23/2015  400000.0         4       2.50         2310   
21594  1523300141   6/23/2014  402101.0         2       0.75         1020   
21595   291310100   1/16/2015  400000.0         3       2.50         1600   
21596  1523300157  10/15/2014  325000.0         2       0.75         1020   

       sqft_lot  floors  waterfront  view  

In [6]:
df1.median()

price            450000.00
bedrooms              3.00
bathrooms             2.25
sqft_living        1910.00
sqft_lot           7618.00
floors                1.50
waterfront            0.00
view                  0.00
condition             3.00
grade                 7.00
sqft_above         1560.00
sqft_basement         0.00
yr_built           1975.00
yr_renovated          0.00
zipcode           98065.00
sqft_living15      1840.00
sqft_lot15         7620.00
dtype: float64

In [7]:
# schools_sdf = gis.content.get('446d756215b945709b9bf7daca9162d5')
schools_df = pd.read_html('https://www.arcgis.com/home/item.html?id=446d756215b945709b9bf7daca9162d5#data')

ValueError: No tables found

In [8]:
def mapping(df):
    budget = eval(input('What is your budget: '))
    schools = input('Are you interested in schools? ')
    budget_df = df.loc[df.price<=budget].copy()
    budget_df.is_copy = None
    gis = GIS()
    if ((schools == 'Yes') or (schools == 'yes')):
        schools_sdf = gis.content.get('446d756215b945709b9bf7daca9162d5')
        pdx_map.add_layer(schools_sdf)
        
    # price = df[df['price']<= budget] 

    df.price.sort_values(ascending= True)
    

    data_sdf = pd.DataFrame.spatial.from_xy(budget_df, 'long','lat')
    pdx_map = gis.map('Seattle,WA')
    pdx_map.basemap = 'dark-gray'
    
    data_sdf.spatial.plot(map_widget=pdx_map, renderer_type = "c", title='Pricing of houses',
                     col='price',
                     cmap='winter',  # matplotlib color map
                     alpha=0.7,
                     size = .5,
                     outline_color=[0,0,0,0]
                     )
    pdx_map.legend = True

    display(pdx_map)

# sns.barplot(x = 'price',y = 'sqft_living',data = budget_df)

In [45]:
mapping(df)

What is your budget: 250000
Are you interested in schools? no


MapView(layout=Layout(height='400px', width='100%'), legend=True)